# PR-027 Glove

곽근봉 님의 [Glove 강의](https://www.youtube.com/watch?v=uZ2GtEe-50E&list=PLlMkM4tgfjnJhhd4wn5aj8fVTYJwIpWkS&index=28) 감사드립니다.

word embedding이 처음이시라면 word2vector.ipynb 부터 보시는 걸 추천드립니다.

Glove를 이용해 간단한 문장을 학습시킵니다.

[tensorflow-glove](https://github.com/GradySimon/tensorflow-glove) 코드를 simple하게 옮겨보았습니다.

논문: https://nlp.stanford.edu/pubs/glove.pdf

## Glove 간단히 살펴보기

간단한 문장 하나를 학습시켜 보면서 glove를 간단히 살펴보겠습니다.

<code>내가 그의 이름을 불러주었을 때, 그는 내게로 와 꽃이 되었다.</code>

데이터는 word2vec 처럼 가공을 합니다.

window_size 가 1일 때의 예입니다.

(word, context)의 형태
```
(그의, 내가)
(그의, 이름을)
(이름을, 그의)
(이름을, 불러주었을)
(내게로, 그는)
(내게로, 와)
...
```

In [1]:
# 현재 케라스에는 glove에서 필요한 cooccurrence matrix를 구하며 깔끔하게 sampling 해주는 함수가 없습니다. (ㅠㅅㅠ)
from collections import defaultdict, Counter

txt = "내가 그의 이름을 불러주었을 때, 그는 내게로 와 꽃이 되었다."
datas = txt.split();

# hyperparameter
vocab_size = 1000    # 총 단어 개수
embed_size = 100     # 임베딩 할 사이즈
ws = 3               # window size
min_occurrences = 0  # 제외할 단어 빈도수

# 각 단어의 빈도수 체크
word_counts = Counter()
word_counts.update(datas)

#### co occurrence matrix 및 sample 구하기
co_occurrence_count = defaultdict(float)
data_length = len(datas)

for i, word in enumerate(datas):
    # 완전 탐색을 하며 left context, word, right context를 얻어냅니다.
    if i < ws:
        l_context = datas[:i]
        r_context = datas[i+1:i+ws+1]
    elif i == data_length - 1:
        l_context = datas[i-ws:i]
        r_context = []
    elif i >= data_length - ws:
        l_context = datas[i-ws:i]
        r_context = datas[i+1:]
    else:
        l_context = datas[i-ws:i]
        r_context = datas[i+1:i+ws+1]
    
    # co occurrence matrix 를 구합니다.
    for i, context_word in enumerate(l_context):
        co_occurrence_count[(word, context_word)] += 1 / (i + 1)
    for i, context_word in enumerate(r_context):
        co_occurrence_count[(word, context_word)] += 1 / (i + 1)
        
### 구한 count를 토대로 words 사전과 cooccurrence matrix를 만듭니다.
# min occurrences가 넘는 words 만 사용합니다.
words = [word for word, count in word_counts.most_common(vocab_size) if count >= min_occurrences]
word_to_id = {word: i for i, word in enumerate(words)}
co_occurrence_matrix = {
    (word_to_id[words[0]], word_to_id[words[1]]): count
    for words, count in co_occurrence_count.items()
    if words[0] in word_to_id and words[1] in word_to_id}

print(words, '\n')
print(word_to_id, '\n')
print(co_occurrence_matrix, '\n')

['그의', '불러주었을', '내게로', '그는', '때,', '이름을', '되었다.', '와', '꽃이', '내가'] 

{'그의': 0, '불러주었을': 1, '내게로': 2, '때,': 4, '그는': 3, '이름을': 5, '내가': 9, '와': 7, '되었다.': 6, '꽃이': 8} 

{(7, 3): 0.5, (4, 7): 0.3333333333333333, (1, 3): 0.5, (9, 1): 0.3333333333333333, (5, 9): 1.0, (2, 8): 0.5, (3, 2): 1.0, (2, 1): 1.0, (6, 2): 1.0, (3, 7): 0.5, (5, 1): 1.0, (7, 2): 0.3333333333333333, (4, 0): 1.0, (1, 2): 0.3333333333333333, (7, 4): 1.0, (9, 0): 1.0, (6, 7): 0.5, (7, 6): 0.5, (1, 5): 0.3333333333333333, (5, 0): 0.5, (0, 4): 0.3333333333333333, (8, 6): 1.0, (3, 5): 1.0, (4, 1): 0.3333333333333333, (0, 1): 0.5, (5, 4): 0.5, (2, 6): 0.3333333333333333, (8, 2): 0.5, (4, 5): 0.5, (1, 4): 1.0, (2, 3): 0.3333333333333333, (1, 9): 1.0, (8, 7): 0.3333333333333333, (9, 5): 0.5, (4, 2): 0.5, (1, 0): 0.5, (5, 3): 0.3333333333333333, (2, 7): 1.0, (8, 3): 1.0, (6, 8): 0.3333333333333333, (3, 4): 0.3333333333333333, (3, 1): 0.5, (3, 8): 0.3333333333333333, (4, 3): 1.0, (0, 9): 1.0, (0, 5): 1.0, (7, 8): 1.0, (2, 4): 0.

이제 모델을 짜 봅시다!

케라스의 [**Embedding layer**](https://keras.io/layers/embeddings/)는 int형 argument들을 vector화 시키는 layer 입니다.

예를 들어
```
[[4], [20]]
```
이 입력되었다면 다음과 같이 output을 반환시켜 줄 수 있습니다.
```
[[0.25, 0.1], [0.6, -0.2]]
```

4 라는 수가 \[0.25, 0.1\] 이라는 벡터로 변한 것이죠.

공식대로 
``` python
word_model_out * context_model_out + b1 + b2 == log(co_occurrence_count)
```
가 되도록 만들어 줍시다

In [2]:
from keras.layers.merge import Dot, Add
from keras.layers.core import Dense, Reshape, Flatten
from keras.layers import Input
from keras.layers.embeddings import Embedding
from keras.models import Sequential, Model
import keras.backend as K

# word model
model_w = Sequential()
model_w.add(Embedding(vocab_size, embed_size, embeddings_initializer='glorot_uniform', input_length=1))
model_w.add(Flatten())

# word bias model
model_wb = Sequential()
model_wb.add(Embedding(vocab_size, 1, embeddings_initializer='glorot_uniform', input_length=1))
model_wb.add(Flatten())
    
# context model
model_c = Sequential()
model_c.add(Embedding(vocab_size, embed_size, embeddings_initializer='glorot_uniform', input_length=1))
model_c.add(Flatten())

# context bias model
model_cb = Sequential()
model_cb.add(Embedding(vocab_size, 1, embeddings_initializer='glorot_uniform', input_length=1))
model_cb.add(Flatten())

# 전체 모델 작성
input_w = Input(shape=(None,))
input_c = Input(shape=(None,))

output_w = model_w(input_w)
output_wb = model_wb(input_w)
output_c = model_c(input_c)
output_cb = model_cb(input_c)

# word model과 context model의 output들을 내적시킵니다.
x = Dot(axes=1)([output_w, output_c])

# bias들을 더합니다.
x = Add()([x, output_wb, output_cb])

model = Model([input_w, input_c], x)
model.compile(loss='mse', optimizer='adam')
model.summary()

Using TensorFlow backend.


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
sequential_1 (Sequential)       (None, 100)          100000      input_1[0][0]                    
__________________________________________________________________________________________________
sequential_3 (Sequential)       (None, 100)          100000      input_2[0][0]                    
__________________________________________________________________________________________________
dot_1 (Dot

In [4]:
import numpy as np
import math

X1 = []
X2 = []
Y = []
for i in co_occurrence_matrix.items():
    X1.append(i[0][0])
    X2.append(i[0][1])
    Y.append(math.log(i[1]))
    
X1 = np.array(X1)
X2 = np.array(X2)
Y = np.array(Y)

In [5]:
# 학습시킵니다.
model.fit([X1, X2], Y,
          epochs=100, 
          verbose=1)

Epoch 1/100
48/48 [==============================] - 0s 2ms/step - loss: 0.5086
Epoch 2/100
48/48 [==============================] - 0s 245us/step - loss: 0.4971
Epoch 3/100
48/48 [==============================] - 0s 187us/step - loss: 0.4867
Epoch 4/100
48/48 [==============================] - 0s 226us/step - loss: 0.4771
Epoch 5/100
48/48 [==============================] - 0s 276us/step - loss: 0.4674
Epoch 6/100
48/48 [==============================] - 0s 227us/step - loss: 0.4582
Epoch 7/100
48/48 [==============================] - 0s 343us/step - loss: 0.4486
Epoch 8/100
48/48 [==============================] - 0s 307us/step - loss: 0.4395
Epoch 9/100
48/48 [==============================] - 0s 265us/step - loss: 0.4306
Epoch 10/100
48/48 [==============================] - 0s 242us/step - loss: 0.4209
Epoch 11/100
48/48 [==============================] - 0s 484us/step - loss: 0.4120
Epoch 12/100
48/48 [==============================] - 0s 239us/step - loss: 0.4030
Epoch 13/100
48

48/48 [==============================] - 0s 270us/step - loss: 1.0728e-04
Epoch 99/100
48/48 [==============================] - 0s 213us/step - loss: 9.8929e-05
Epoch 100/100
48/48 [==============================] - 0s 359us/step - loss: 9.1019e-05


실제로 사용하는 부분은 model_w의 embedding layer 입니다.

임베딩하는 방식이 중요한 것이지요.

model_w 에 벡터화 시키고 싶은 수를 넣으면 결과가 나올 것입니다.

In [6]:
print(model_w.predict(np.array([3])))

[[-0.04126682  0.02041662  0.07496435 -0.07006773  0.06896963  0.05239534
  -0.06739062 -0.02732384  0.07687686 -0.10646415  0.08526754 -0.11001399
   0.16387607  0.11731478 -0.09811315 -0.16818208 -0.10618692  0.10006183
  -0.07625895 -0.0956468  -0.16605097 -0.0733607   0.05235191  0.05943675
  -0.05344569 -0.16186695 -0.06868561 -0.07074318  0.13461851  0.07791412
  -0.02670521  0.17489552  0.10048576  0.05107064 -0.15255289 -0.07233072
  -0.04385498 -0.10624173 -0.07228147  0.10215108 -0.12747085 -0.01945782
   0.15561906 -0.18081762 -0.00810576 -0.02820941  0.06056061  0.15553233
   0.10657778 -0.14834158 -0.11995964 -0.02345999  0.16221862  0.00799892
   0.10882306  0.13122565 -0.11651555 -0.12902538  0.04407527 -0.01213415
   0.02516619 -0.16075747  0.18391143  0.01563227  0.05561586  0.14834054
   0.15524881 -0.16697121 -0.14766945  0.02491257  0.11908259 -0.0445068
  -0.09575619 -0.11983696  0.09935905 -0.00665964  0.16705258  0.17732404
   0.17635936  0.08531837  0.13246793  

사전 학습된 glove를 사용해 보고 싶으시다면 아래 링크를 참고하시면 좋겠습니다.

https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html

코드에서 문제가 되는 부분이 있다면 꼭 연락 주세요

## Contact me
케라스를 사랑하는 개발자 입니다.

질문, 조언, contribtuion 등 소통은 언제나 환영합니다.

Anthony Kim(김동현) : artit.anthony@gmail.com
